In [10]:
import pandas as pd
import json
import base64

# Daten zunächst zusammenführen (CSV mit Daten der JSON verknüpfen)

In [12]:
# JSON Sammlung aller Transaktionen laden:
json_file_path = "./LNDataRaw/all_payments.json"
with open(json_file_path, "r", encoding="utf-8") as file:
    json_data = json.load(file)

# Erstellen eines Dictionarys {payment_hash: (attempt_time_ns, resolve_time_ns)}
payment_times = {}
for payment in json_data:
    payment_hash = payment["payment_hash"]
    
    # Falls es mindestens einen HTLC-Versuch gibt, nehme die ersten Attempt- und Resolve-Zeiten
    if payment["htlcs"]:
        attempt_time_ns = payment["htlcs"][0].get("attempt_time_ns", 0)
        resolve_time_ns = payment["htlcs"][0].get("resolve_time_ns", 0)
        payment_times[payment_hash] = (attempt_time_ns, resolve_time_ns)

In [18]:
# Mapping der Base64-codierten Payment Hashes in der CSV auf Hex-codierte in der JSON-Datei
import base64

def decode_payment_hash(b64_hash):
    """ Decodiert den base64-Hash aus der CSV in ein hex-Format für den JSON-Abgleich """
    return base64.b64decode(b64_hash).hex()

df_csv = pd.read_csv("./LNDataRaw/all_transactions.csv")

# Anwenden auf die CSV-Spalte "Payment Hash"
df_csv["Decoded Payment Hash"] = df_csv["Payment Hash"].apply(decode_payment_hash)

# `attempt_time_ns` und `resolve_time_ns` in die CSV einfügen
df_csv["Attempt Time (ns)"] = df_csv["Decoded Payment Hash"].map(lambda x: payment_times.get(x, (0, 0))[0])
df_csv["Resolve Time (ns)"] = df_csv["Decoded Payment Hash"].map(lambda x: payment_times.get(x, (0, 0))[1])

# Unnötige Spalte entfernen
df_csv.drop(columns=["Decoded Payment Hash"], inplace=True)

df_csv.head()


,Payment Hash,Amount (sats),Num Hops,Fee (sats),Local Start (unix),Local End (unix),Local Duration (s),Attempt Time (ns),Resolve Time (ns),LN Duration (s)
0,Dqib5PeAW4C570mdvPKg21J+GO330mB6KWn8Q+KVcCI=,100000,3,98,1737735151,1737735154,3.818,1737735151613859688,1737735154437257619,0.0
1,KWItLScHRNKeyno+IJ5nN2aID/BqNPLzH+4AEtxUxA8=,100000,3,98,1737735557,1737735581,24.147,1737735557819848280,1737735581325330395,0.0
2,EbdvXElr0FeSlN7Sc/CsQQeUp5RmNYfxXhM3CCSKUxc=,100000,3,98,1737736022,1737736030,7.315,1737736023039444961,1737736029577915367,0.0
3,PhvPxSW5TAS29RpN6klDRd9GplIbrhklBFzgGH29NM0=,100000,3,98,1737736486,1737736496,9.514,1737736487061399757,1737736495926806466,0.0
4,ggIyGqPVYlMFb/j2JJkAisukjheHPu58c63MHQu36oU=,100000,3,98,1737736906,1737736928,22.587,1737736906484687988,1737736928392144332,0.0


Nachdem Attempt Time und Resolve Time stimmen kann jetzt die LN Duration durch Differenzbildung ermittelt werden um den Datensatz zu finalisieren:

In [28]:
# Umwandlung der AttemptTime und Resolve Time in integer:
df_csv["Attempt Time (ns)"] = pd.to_numeric(df_csv["Attempt Time (ns)"], errors="coerce")
df_csv["Resolve Time (ns)"] = pd.to_numeric(df_csv["Resolve Time (ns)"], errors="coerce")

# Berechnung der LN-Dauer in Sekunden
df_csv["LN Duration (s)"] = (df_csv["Resolve Time (ns)"] - df_csv["Attempt Time (ns)"]) / 1e9

df_csv.head()


,Payment Hash,Amount (sats),Num Hops,Fee (sats),Local Start (unix),Local End (unix),Local Duration (s),Attempt Time (ns),Resolve Time (ns),LN Duration (s)
0,Dqib5PeAW4C570mdvPKg21J+GO330mB6KWn8Q+KVcCI=,100000,3,98,1737735151,1737735154,3.818,1737735151613859688,1737735154437257619,2.823398
1,KWItLScHRNKeyno+IJ5nN2aID/BqNPLzH+4AEtxUxA8=,100000,3,98,1737735557,1737735581,24.147,1737735557819848280,1737735581325330395,23.505482
2,EbdvXElr0FeSlN7Sc/CsQQeUp5RmNYfxXhM3CCSKUxc=,100000,3,98,1737736022,1737736030,7.315,1737736023039444961,1737736029577915367,6.538470
3,PhvPxSW5TAS29RpN6klDRd9GplIbrhklBFzgGH29NM0=,100000,3,98,1737736486,1737736496,9.514,1737736487061399757,1737736495926806466,8.865407
4,ggIyGqPVYlMFb/j2JJkAisukjheHPu58c63MHQu36oU=,100000,3,98,1737736906,1737736928,22.587,1737736906484687988,1737736928392144332,21.907456
